In [36]:
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

template = """You are an assistant that answers the following question correctly and honestly: {question}\n\n"""
prompt_template = PromptTemplate(input_variables=["question"], template=template)

question_chain = LLMChain(llm=llm, prompt=prompt_template)

question_chain.run("what is the latest fast and furious movie?")

'The latest Fast and Furious movie is "F9: The Fast Saga," released in 2021.'

In [45]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
TOP_N_RESULTS = 10

def top_n_results(query):
    return search.results(query, TOP_N_RESULTS)

tool = Tool(
    name = "Google Search",
    description="Search Google for recent results.",
    func=top_n_results
)

query = "Qual é o último filme de velozes e furiosos lançado?"

results = tool.run(query)

for result in results:
    print(result["title"])
    print(result["link"])
    print(result["snippet"])
    print("-"*50)

"Velozes e Furiosos 10": o que sabemos sobre o "começo do fim" da ...
https://www.correiodopovo.com.br/arteagenda/velozes-e-furiosos-10-o-que-sabemos-sobre-o-come%C3%A7o-do-fim-da-franquia-1.1034073
May 17, 2023 ... Parecia que esse momento não chegaria nunca. Finalmente chegou a semana de estreia do novo "Velozes e Furiosos". O 10º filme da franquia que ...
--------------------------------------------------
'Velozes e furiosos': Veja lista com todos os 11 filmes da franquia ...
https://g1.globo.com/pop-arte/cinema/noticia/2023/05/18/velozes-e-furiosos-veja-lista-com-todos-os-11-filmes-da-franquia-do-pior-para-o-melhor.ghtml
May 18, 2023 ... 3 - 'Velozes e furiosos 7' (2015) ... "Velozes e furiosos 7" é o último com a participação de Walker, que morreu dois anos antes em um acidente de ...
--------------------------------------------------
Velozes & Furiosos 10 - Filme 2023 - AdoroCinema
https://www.adorocinema.com/filmes/filme-242871/
No décimo filme da série Velozes & Furiosos e últi

In [60]:
import newspaper

pages_content = []

for result in results:
	try:
		article = newspaper.Article(result["link"])
		article.download()
		article.parse()
		if len(article.text) > 0:
			pages_content.append({ "url": result["link"], "text": article.text })
	except:
		continue

In [61]:
pages_content

[{'url': 'https://www.correiodopovo.com.br/arteagenda/velozes-e-furiosos-10-o-que-sabemos-sobre-o-come%C3%A7o-do-fim-da-franquia-1.1034073',
  'text': 'publicidade\n\nParecia que esse momento não chegaria nunca. Finalmente chegou a semana de estreia do novo "Velozes e Furiosos". O 10º filme da franquia que tem Vin Diesel como estrela promete ser maior e melhor que qualquer outro já lançado. E olha que eles já fizeram a proeza de lançar um carro no espaço.\n\n"Velozes e Furiosos 10" estreia nos cinemas nesta quinta-feira, 18, nas principais salas do País e seguirá um enredo que começou em "Velozes e Furiosos 5: Operação Rio".\n\nO que sabemos sobre "Velozes e Furiosos 10" ?\n\nDe cara, o filme foi classificado como o mais caro da franquia, com orçamento estimado em US$ 340 milhões, sendo o terceiro filme mais caro da história do cinema. Segundo o The Wrap o valor se justifica pelo alto salário do elenco, a troca de diretor e as medidas de segurança contra a covid-19. Para se ter uma ide

In [62]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

text_splitter = RecursiveCharacterTextSplitter(chunk_size=6000, chunk_overlap=100)

docs = []
for d in pages_content:
	chunks = text_splitter.split_text(d["text"])
	for chunk in chunks:
		new_doc = Document(page_content=chunk, metadata={ "source": d["url"] })
		docs.append(new_doc)

In [63]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

docs_embeddings = embeddings.embed_documents([doc.page_content for doc in docs])
query_embedding = embeddings.embed_query(query)

In [64]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_top_k_indices(list_of_doc_vectors, query_vector, top_k):
  # convert the lists of vectors to numpy arrays
  list_of_doc_vectors = np.array(list_of_doc_vectors)
  query_vector = np.array(query_vector)

  # compute cosine similarities
  similarities = cosine_similarity(query_vector.reshape(1, -1), list_of_doc_vectors).flatten()

  # sort the vectors based on cosine similarity
  sorted_indices = np.argsort(similarities)[::-1]

  # retrieve the top K indices from the sorted list
  top_k_indices = sorted_indices[:top_k]

  return top_k_indices

top_k = 1
best_indexes = get_top_k_indices(docs_embeddings, query_embedding, top_k)
best_k_documents = [doc for i, doc in enumerate(docs) if i in best_indexes]

In [65]:
best_k_documents

[Document(page_content='Todos prontos para um dos maiores lançamentos de 2023? Velozes e Furiosos 10 (Fast X) já está acelerando em direção aos cinemas de todo o mundo e fará sua estreia oficial no dia 19 de maio. Dirigida por Louis Leterrier, a obra será a penúltima da franquia de ação, que teve início em 2001, com Velozes e Furiosos (The Fast and the Furious), e promete surpreender os fãs com novas narrativas de tirar o fôlego, além de novos personagens.\n\nAbaixo, saiba tudo sobre Velozes e Furiosos 10 enquanto aguarda pela estreia da produção!\n\nVelozes e Furiosos 10: trailer e sinopse\n\nPor enquanto, a trama principal de Velozes e Furiosos 10 ainda é um mistério, sem grandes detalhes revelados. A sinopse oficial diz apenas o seguinte: "Dom Toretto e sua família devem lidar com o adversário mais letal que já enfrentaram. Alimentada pela vingança, uma ameaça terrível emerge das sombras do passado para destruir o mundo de Dom e de todos que ele ama".\n\nO filme de ação terá 2h21min

In [67]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

response = chain({"input_documents": best_k_documents, "question": query}, return_only_outputs=True)

response_text, response_sources = response["output_text"].split("SOURCES:")
response_text = response_text.strip()
response_sources = response_sources.strip()

print(f"Answer: {response_text}")
print(f"Sources: {response_sources}")

Answer: O último filme de Velozes e Furiosos lançado foi Velozes e Furiosos 10 (Fast X) em 19 de maio de 2023.
Sources: https://www.tecmundo.com.br/minha-serie/263683-velozes-furiosos-10-veja-data-lancamento-trailer-sinopse-filme.htm
